<a href="https://colab.research.google.com/github/oyyarko/deeplearning_cnn/blob/master/text_generation_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
tensorflow.__version__

Using TensorFlow backend.


'2.2.0-rc1'

In [3]:
!wget https://www.gutenberg.org/files/11/11-0.txt

--2020-03-29 09:10:37--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174481 (170K) [text/plain]
Saving to: ‘11-0.txt’

11-0.txt            100%[===================>] 170.39K   865KB/s    in 0.2s    

2020-03-29 09:10:37 (865 KB/s) - ‘11-0.txt’ saved [174481/174481]



In [0]:
filename = "wonderland.txt"

#with open(filename, 'r') as f:
  #  raw_text = f.read()
    #raw_text = raw_text.lower()
    
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [5]:
chars = sorted(list(set(raw_text)))
chars_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_cahr = dict((i, c) for i, c in enumerate(chars))
chars_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 'ù': 58,
 '—': 59,
 '‘': 60,
 '’': 61,
 '“': 62,
 '”': 63}

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Text: ", n_chars, "\nVocab: ", n_vocab)

Text:  164201 
Vocab:  64


In [7]:
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([chars_to_int[char] for char in seq_in])
    dataY.append(chars_to_int[seq_out])

n_patterns = len(dataX)
print("Total Pattern: ", n_patterns)

Total Pattern:  164101


In [0]:
X = np.reshape(dataX, (n_patterns, seq_length, 1))
X = X/float(n_vocab)
y = np_utils.to_categorical(dataY)

## LSTM model

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
Total params: 805,952
Trainable params: 805,952
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
filepath = "weights-improvement-bigger-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [28]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
 105/2565 [>.............................] - ETA: 1:10 - loss: 1.3076

KeyboardInterrupt: ignored

## **Generating Text with LSTM network**

In [0]:
filename = "weights-improvement-bigger-50-1.3423.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("seed: ")
print("\"", ''.join([int_to_cahr[value] for value in pattern]), "\"")

seed: 
"  “come away, my dears! it’s high time you were all in
bed!” on various pretexts they all moved off,  "


In [31]:
len(pattern)

100

In [32]:
import sys
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_cahr[index]
    seq_in = [int_to_cahr[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("Done")

and the white rabbit were the white rabbit, and the white rabbit was so donterseti the white rabbit, and the white rabbit was so donter to tee the words “dro it was all the lobsters, aut ie would not de mooe the world better to tell her heads  the suejseed of her fead would be wery look and spoeersed to the toot of the words would be wery looking her armsher with the door, and the white rabbit was so done of the had fone the white rabbit, and the white rabbit was not a little shated wpuld be were the words “dro it with the dance.” 
“i don’t know what a mouse of the door,” said the king. 
“i don’t think it would be a large diamce,” said the king, “i’ve seen to the words!”

“i don’t know what a mouse of the dorrtastion,” she mock turtle said to herself, “i’ve seen to the book,” shought alice, “i’ve seen to the words!”

“i don’t know what a mouse of the dorrtastion,” she mock turtle said to herself, “i’ve seen to the book,” shought alice, “i’ve seen to the words!”

“i don’t know what a mo